# Using Large Language Models (LLMs)

This tutorial will give us an introduction to LLMs: How to use them effectively, what can we do with them, and we will explore a bit how everything works in the background!

In [ ]:
OPENAI_API_KEY = "insert-key-here"
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## 1. Using an LLM: The very basics

Current LLMs are big models, so big that typically you need specialized and expensive hardware to run them (although this is [changing very rapidly](https://medium.com/@gabrielrodewald/running-models-with-ollama-step-by-step-60b6f6125807)!!).
Also, the state-of-the-art models are not openly available, and companies provide them over an Application Programming Interface (API) so that we can have access to these amazing models without having to worry about all the [engineering details](https://github.com/mlabonne/llm-course?tab=readme-ov-file#1-running-llms) of loading and running inference with one of these huge models.

#### Here we will see how to communicate with one of these APIs

In [ ]:
# Let's make a simple request to OpenAI directly

import json
import requests

url = "https://api.openai.com/v1/responses"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {OPENAI_API_KEY}"
}
data = {
    "model": "gpt-4o-mini",
    "input": "Say hi to all the students in Zadar!"
}

response = requests.post(url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

In [ ]:
# The response itself can be extracted from the response object
response_json = response.json()
answer = (
    response_json
    .get("output", [{}])[0]
    .get('content', [{}])[0]
    .get("text", {})
)

print(answer)

## 2. LangChain

#### Langchain is a framework for developing applications powered by language models.


It aims to:

- Be data-aware: connect a language model to other sources of data

- Be agentic: allow a language model to interact with its environment

In [ ]:
import os
from utils import *
from langchain import (
    PromptTemplate,
    chains
)
from langchain.chat_models import init_chat_model


### Language Models

In [ ]:
# Let's now initialize a language model.
llm = init_chat_model(
    "gpt-4o-mini",
    model_provider="openai"
)

from langchain_core.messages import HumanMessage
ai_message = llm.invoke([HumanMessage("What functional groups are present in this molecule? c1(Br)ccc(CC(=O)Cl)cc1.")])
ai_message

In [ ]:
ai_message.content

### Prompt Templates

In [ ]:
# First, define a prompt template
prompt_template = PromptTemplate(
    input_variables = ["smiles"],
    template = (
        "What functional groups are present in this molecule? {smiles}. "
    )
)

In [ ]:
smiles = "c1(Br)ccc(CC(=O)Cl)cc1"

cdk(smiles)
prompt_template.format(smiles=smiles)

### Putting it together: Chains

In [ ]:
fg_chain = chains.LLMChain(
    prompt = prompt_template,
    llm = llm
)

cdk(smiles)
fg_chain.invoke(smiles).get('text')

In [ ]:
smiles_2 = "c1ccc(-c2cncc(C3CCCCC3)c2)cc1"
fg_chain.invoke(smiles_2).get('text')

### Improving prompting: Formatting, in-context learning, etc.

In [ ]:
prompt_template_2 = PromptTemplate(
    input_variables = ["smiles"],
    template = (
        "You are an expert chemist and your task is to identify the functional groups of the given molecules."\
        "You should give the name and the SMILES of each functional group. Begin!"\
        "Input: Brc1cncc(C2CCCCC2)c1"\
        "Output: 1. Halogen (Br)\n 2. Pyridine (c1cnccc1)\n 3. Cyclohexane (C2CCCCC2)"\
        "Input: c1ccccc1CC(=O)Cl"\
        "Output: 1. Phenyl (c1ccccc1)\n 2. Carbonyl (C=O)\n 3. Acyl halide (C(=O)Cl)\n 4. Halogen (Cl)"\
        "Input: {smiles}"\
        "Output:"
    )
)

fg_chain_new = chains.LLMChain(
    prompt = prompt_template_2,
    llm = llm
)
def fcs(smiles):
    cdk(smiles)
    print(fg_chain_new.invoke(smiles).get('text'))

In [ ]:
smiles_list = [
    smiles,
    smiles_2,
    "C/C=C/C(=O)I",
    "C/C=C/C(=O)S"
]

for s in smiles_list:
    fcs(s)

## 3. Applications: Data Extraction

LLMs are good with language. What if they could help us turn unstructured data (like text in papers) into structured data (like a tabular database or a knowledge graph!)



In [ ]:
from typing import Optional, List
from pydantic import BaseModel, Field

# Define a data model in Pydantic
class Synthesis(BaseModel):
    """Data Model for an organic synthesis."""

    product: str = Field(description="The main product of the reaction")
    reactants: List[str] = Field(description="The reactants used in the synthesis")
    temperature: Optional[int] = Field(description="Temperature in degrees Celsius")


structured_llm = llm.with_structured_output(Synthesis)

In [ ]:
# Procedure taken from https://www.orgsyn.org/demo.aspx?prep=v101p0488
text = """(1S, 3R)-3-Methylcyclohexan-1-ol (2). A single-necked (24/40 joint) 500 mL round-bottom flask was equipped with a Teflon-coated magnetic stir bar (2.5 x 0.5 cm, pill-shaped). To the flask was added (-)-isopulegol (1) (8.5 mL, 7.7 g, 50 mmol, 1.0 equiv) (Notes 2 and 3) via syringe. Then, MeOH (200 mL, 0.25 M) (Note 4) is added at 23 ℃ to the flask from a graduated cylinder. The mixture was stirred (500 rpm) briefly at room temperature until a homogenous mixture is achieved. The flask was then placed in a saturated dry ice/Acetone bath (400 mL) (Notes 5) (1000 mL dewar) and cooled to -78 ℃ while open to the air. Ozone (Notes 6 and 7) was bubbled through the clear colorless solution (Figure 1A) for 50-100 minutes (Note 8), until complete consumption of the starting material had occurred (Figure 1B). Complete consumption can be indicated by a faint blue color (Note 9). The solution was then sparged with argon (tank pressure 11 psi) through a 16 G needle submerged in the reaction for 20 minutes to expel the excess ozone (Figure 1C)."""
print(text)

In [ ]:
synthesis = structured_llm.invoke(f"Extract the synthesis information from the following text: {text}")
synthesis

In [ ]:
synthesis.product

In [ ]:
synthesis.temperature

## 4. Applications: Agents

<img src="agents.webp" width=400 display="block">

An agent is "something that produces or is capable of producing an effect : an active or efficient cause".

In the context of LLMs, let's consider an agent as **an entity that ---given a goal--- will plan, decide and act on its environment in order to reach its goal**.

We'll be using LangChain to build agents in this tutorial, but there's plenty of frameworks that have appeared over the years.


### Agents:

We would like to integrate other things:
- User input
- External knowledge sources
- External tools
- Memory storage
- ...

In [ ]:
# Let's define a tool!
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaAPIWrapper()
wikipedia.run("Atorvastatin")

In [ ]:
from langchain.agents import initialize_agent, Tool

# Define a toolset 
toolset = [
    Tool(
        name="wikipedia search",
        func=wikipedia.run,
        description="Useful to get accurate information from wikipedia."
    )
]

# Define an agent
jamesbond = initialize_agent(
    toolset,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [ ]:
jamesbond.invoke(f"What are the functional groups of {smiles}")

# Probably not the most useful tool for this task...

In [ ]:
from utils import FuncGroups

toolset += [FuncGroups()]

# Define an agent
jamesbond = initialize_agent(
    toolset,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [ ]:
jamesbond.invoke(f"What are the functional groups of {smiles}")

# What's happening behind the courtain?

In [ ]:
prompt = jamesbond.agent.create_prompt(toolset)
james_prompt = jamesbond.agent.llm_chain.prompt

print(james_prompt.input_variables)

In [ ]:
print(james_prompt.template)

# A more useful agent

In [ ]:
from utils import Query2SMILES

toolset += [Query2SMILES()]

llm = init_chat_model(
    "gpt-4o",
    model_provider="openai"
)

# Define an agent
jamesbond = initialize_agent(
    toolset,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

final_answer = jamesbond.run(
    "Find the functional groups of cafeine, p-bromobenzaldehyde, and cyclosarin, "
    "then find what they have in common, and find some information on about it in wikipedia."
)

In [ ]:
print(final_answer)